In [ ]:
#!conda install -c conda-forge lightgbm
#!pip install optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import sys
import os
import re
import random

from time import time
from tqdm import tqdm

from contextlib import contextmanager
import lightgbm as lgb
#import optuna.integration.lightgbm as opt_lgb

In [ ]:
#上限表示数を拡張
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 200)

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

In [ ]:
INPUT = "/content/drive/MyDrive/nishika/" # 所望のディレクトリに変更してください。
train_df = pd.read_csv(os.path.join(INPUT, "train.csv"))
test_df = pd.read_csv(os.path.join(INPUT, "test.csv"))
submission_df = pd.read_csv(os.path.join(INPUT, "sample_submission.csv"))

In [ ]:
df_1_tr = pd.read_csv("/content/drive/MyDrive/nishika/output/train_SwinTransformerLarge384_V2Large256.csv")
df_1_ts = pd.read_csv("/content/drive/MyDrive/nishika/output/test_SwinTransformerLarge384_V2Large256.csv")

In [ ]:
df_2_tr = pd.read_csv("/content/drive/MyDrive/nishika/output/train_efb2_rinnabase.csv")
df_2_ts = pd.read_csv("/content/drive/MyDrive/nishika/output/test_efb2_rinnabase.csv")

In [ ]:
df_2_tr = df_2_tr.drop(df_2_tr.columns[352:], axis=1)

In [ ]:
df_2_ts = df_2_ts.drop(df_2_ts.columns[352:], axis=1)

In [ ]:
df_3_tr = pd.read_csv("/content/drive/MyDrive/nishika/output/train_swinv2base256_ginza510electra.csv")
df_3_ts = pd.read_csv("/content/drive/MyDrive/nishika/output/test_swinv2base256_ginza510electra.csv")

In [ ]:
df_4_tr = pd.read_csv("/content/drive/MyDrive/nishika/output/train_efv2b2_resnet152_jaginza.csv")
df_4_ts = pd.read_csv("/content/drive/MyDrive/nishika/output/test_efv2b2_resnet152_jaginza.csv")

In [ ]:
df_4_tr = df_4_tr.drop(df_4_tr.columns[352:], axis=1)

In [ ]:
df_4_ts = df_4_ts.drop(df_4_ts.columns[352:], axis=1)

In [ ]:
#swinL384,swinV2L256,Roberta-text,efb2
df_tmp_tr_1 = pd.concat([df_1_tr,df_2_tr],axis=1)
df_tmp_ts_1 = pd.concat([df_1_ts,df_2_ts],axis=1)

In [ ]:
#ginza510-electra-base-text,swinv2base256
df_tmp_tr_2 = pd.concat([df_tmp_tr_1,df_3_tr],axis=1)
df_tmp_tr_2 = pd.concat([df_1_tr,df_3_tr],axis=1)
df_tmp_ts_2 = pd.concat([df_tmp_ts_1,df_3_ts],axis=1)
df_tmp_ts_2 = pd.concat([df_1_ts,df_3_ts],axis=1)

In [ ]:
#efficientnetv2b2,resnet152,jaginza(ginza_electra_text)
df_tmp_tr_3 = pd.concat([df_tmp_tr_2,df_4_tr],axis=1)
df_tmp_ts_3 = pd.concat([df_tmp_ts_2,df_4_ts],axis=1)

In [ ]:
#train_tmp = df_tmp_tr_3.drop(["is_laugh","odai_photo_file_name"], axis=1)

In [ ]:
#test_tmp = df_tmp_ts_3.drop(["odai_photo_file_name"], axis=1)

In [ ]:
#df_5_tr = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_image_beit.csv")
#df_5_ts = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_image_beit.csv")

In [ ]:
#df_5_tr = df_5_tr.drop(["Unnamed: 0","odai_photo_file_name","id", "text","is_laugh"], axis=1)
#df_5_ts = df_5_ts.drop(["Unnamed: 0","odai_photo_file_name","id", "text"], axis=1)

In [ ]:
#train_tmp = pd.concat([train_emb,df_5_tr],axis=1)
#train_tmp = pd.concat([train_emb,df_4_tr],axis=1)
#test_tmp = pd.concat([test_emb,df_5_ts],axis=1)
#test_tmp = pd.concat([test_emb,df_4_ts],axis=1)

In [ ]:
#df_6_tr = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_rinna_clip_big_batch.csv")
#df_6_ts = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_rinna_clip_bigbatch.csv")

In [ ]:
#df_6_tr = df_6_tr.drop(["img_path","Unnamed: 0","odai_photo_file_name","id", "text","is_laugh"], axis=1)
#df_6_ts = df_6_ts.drop(["img_path","Unnamed: 0","odai_photo_file_name","id", "text"], axis=1)

In [ ]:
#df_6_tr = df_6_tr.drop(df_6_tr.columns[0:1030], axis=1)

In [ ]:
#df_6_ts = df_6_ts.drop(df_6_ts.columns[0:1029], axis=1)

In [ ]:
#train_tmp_2 = pd.concat([train_tmp,df_6_tr],axis=1)
#test_tmp_2 = pd.concat([test_tmp,df_6_ts],axis=1)

In [ ]:
#cos_tr = pd.read_csv('/content/drive/MyDrive/nishika/output/tsne_cos_sim_tr_df.csv')
#cos_ts = pd.read_csv('/content/drive/MyDrive/nishika/output/tsne_cos_sim_ts_df.csv')

In [ ]:
#train = pd.concat([train_tmp_2,train_df],axis=1)
#train = pd.concat([train_tmp,cos_tr],axis=1)
#test = pd.concat([test_tmp_2,test_df],axis=1)
#test = pd.concat([test_tmp,cos_ts],axis=1)

In [ ]:
#tr_debv3 = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_debertav3.csv")
#ts_debv3 = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_debertav3.csv")

In [ ]:
#tr_debv3 = tr_debv3.drop(["Unnamed: 0","is_laugh","id","text","odai_photo_file_name"], axis=1)

In [ ]:
#ts_debv3 = ts_debv3.drop(["Unnamed: 0","id","text","odai_photo_file_name"], axis=1)

In [ ]:
#tr_debv2 = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_debertav2.csv")
#ts_debv2 = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_debertav2.csv")

In [ ]:
#tr_debv2 = tr_debv2.drop(["Unnamed: 0","is_laugh","id","text","odai_photo_file_name"], axis=1)
#ts_debv2 = ts_debv2.drop(["Unnamed: 0","id","text","odai_photo_file_name"], axis=1)

In [ ]:
#tr_deb = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_deberta.csv")
#ts_deb = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_deberta.csv")

In [ ]:
#tr_deb = tr_deb.drop(["Unnamed: 0","is_laugh","id","text","odai_photo_file_name"], axis=1)
#ts_deb = ts_deb.drop(["Unnamed: 0","id","text","odai_photo_file_name"], axis=1)

In [ ]:
#tr_tmp_db1= pd.concat([tr_deb,tr_debv2],axis=1)
#ts_tmp_db1 = pd.concat([ts_deb,ts_debv2],axis=1)

In [ ]:
#tr_tmp_db2= pd.concat([tr_tmp_db1,tr_debv3],axis=1)
#ts_tmp_db2 = pd.concat([ts_tmp_db1,ts_debv3],axis=1)

In [ ]:
#train_tmp_2 = pd.concat([tr_tmp_db1,tr_tmp_db2],axis=1)
#test_tmp_2 = pd.concat([ts_tmp_db1,ts_tmp_db2],axis=1)

In [ ]:
#tr_tmpo = pd.concat([train_tmp,tr_deb],axis=1)
#ts_tmpo = pd.concat([test_tmp,ts_deb],axis=1)

In [ ]:
#train_tmpo = pd.concat([df_2_tr,df_4_tr],axis=1)
#test_tmpo = pd.concat([df_2_ts,df_4_ts],axis=1)

In [ ]:
#train_tmpo = pd.concat([train_tmpo,df_6_tr],axis=1)
#test_tmpo = pd.concat([test_tmpo,df_6_ts],axis=1)

In [ ]:
#train_tmp = pd.concat([train_tmpo,train_tmp_2],axis=1)
#test_tmp = pd.concat([test_tmpo,test_tmp_2],axis=1)

In [ ]:
train = pd.concat([df_tmp_tr_3,train_df],axis=1)
test = pd.concat([df_tmp_ts_3,test_df],axis=1)

In [ ]:
print(train)

In [ ]:
train["text_len"] = train["text"].str.len()
test["text_len"] = test["text"].str.len()

In [ ]:
# 学習データと評価データに分割します
train, valid = train_test_split(train, test_size=0.2, random_state=42, stratify=train["is_laugh"])

train_y = train["is_laugh"]
train_x = train.drop(["odai_photo_file_name","is_laugh","id","text"], axis=1)

valid_y = valid["is_laugh"]
valid_x = valid.drop(["odai_photo_file_name","is_laugh","id","text"], axis=1)

test_x = test.drop(["odai_photo_file_name","id","text"], axis=1)

In [ ]:
print(train_x.shape)
print(valid_x.shape)

(19969, 2146)
(4993, 2146)


In [ ]:

train_x = train_x.T.drop_duplicates().T
valid_x = valid_x.T.drop_duplicates().T 
test_x = test_x.T.drop_duplicates().T 


In [ ]:
"""
dtrain = opt_lgb.Dataset(train_x, train_y)
deval = opt_lgb.Dataset(valid_x, valid_y, reference=dtrain)

best_params, history = {}, []
param = {
        'objective': 'binary',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }

opt = opt_lgb.train(param, 
                 dtrain,
                 valid_sets=deval,
                 num_boost_round=10000,
                 early_stopping_rounds=100)
"""

In [ ]:
#最適化したパラメータを表示する。
#best_params = opt.params
#print(best_params)
"""
params = {  
    "n_estimators": 20000,
    "objective": 'binary',
    "learning_rate": 0.01,
    'lambda_l1': 8.553323365580143, 
    'lambda_l2': 0.00024389221237366783, 
    'num_leaves': 240, 
    'feature_fraction': 0.6839999999999999, 
    'bagging_fraction': 0.6378550454605715, 
    'bagging_freq': 1,
    "random_state": 71,
    "n_jobs": -1,
    "importance_type": "gain",
    'colsample_bytree': .8,
    "reg_lambda": 5,
    "max_depth":9,
    'min_child_samples': 20
    }

lgtrain = lgb.Dataset(train_x, train_y)
lgvalid = lgb.Dataset(valid_x, valid_y)

lgb_clf = lgb.train(
    params,
    lgtrain,
    num_boost_round=10000,
    valid_sets=[lgtrain, lgvalid],
    valid_names=['train','valid'],
    early_stopping_rounds=100,
    verbose_eval=100
)
"""

In [ ]:

params = {  
    "n_estimators": 20000,
    "objective": 'binary',
    "learning_rate": 1e-5,
    'lambda_l1': 8.553323365580143, 
    'lambda_l2': 0.00024389221237366783, 
    'num_leaves': 240, 
    'feature_fraction': 0.6839999999999999, 
    'bagging_fraction': 0.6378550454605715, 
    'bagging_freq': 1,
    "random_state": 71,
    "n_jobs": -1,
    "importance_type": "gain",
    'colsample_bytree': .8,
    "reg_lambda": 5,
    "max_depth":9,
    'min_child_samples': 20
    }

lgtrain = lgb.Dataset(train_x, train_y)
lgvalid = lgb.Dataset(valid_x, valid_y)

lgb_clf = lgb.train(
    params,
    lgtrain,
    num_boost_round=10000,
    valid_sets=[lgtrain, lgvalid],
    valid_names=['train','valid'],
    early_stopping_rounds=100,
    verbose_eval=100
)

In [ ]:
# 特徴量の重要度を可視化。
lgb.plot_importance(lgb_clf, figsize=(15,20), max_num_features=100, importance_type='gain')
plt.tight_layout()
plt.show()

In [ ]:
# 評価指標はlog lossだが、accuracyも見てみる

val_pred = lgb_clf.predict(valid_x, num_iteration=lgb_clf.best_iteration)
val_pred_max = np.round(lgb_clf.predict(valid_x)).astype(int)  # クラスに分類
accuracy = sum(valid_y == val_pred_max) / len(valid_y)
print(accuracy)


default
0.6214700580813138
d lr 0.01 depth9
0.6328860404566393

In [ ]:
_conf_options = {"normalize": None,}
_plot_options = {
        "cmap": "Blues",
        "annot": True
    }

conf = confusion_matrix(y_true=valid_y,
                        y_pred=val_pred_max,
                        **_conf_options)

fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(conf, ax=ax, **_plot_options)
ax.set_ylabel("Label")
ax.set_xlabel("Predict")

In [ ]:
test_pred = lgb_clf.predict(test_x, num_iteration=lgb_clf.best_iteration)

In [ ]:
submission_df["is_laugh"] = test_pred
submission_df.head()

In [ ]:
submission_df.to_csv(('sub.csv'), index=False)